# Text generation with RNN

In this lab, we are going to generate text with RNNs.

We'll try to have a RNN learning the *fables de la Fontaine*.

Lets load into variable the *Fables*:

In [5]:
with open('') as f:
    text = f.read()

print(text)

À Monseigneur le Dauphin
 
Je chante les héros dont Ésope est le père,
Troupe de qui l'histoire, encor que mensongère,
Contient des vérités qui servent de leçons.
Tout parle en mon ouvrage, et même les poissons.
Ce qu'ils disent s'adresse à tous tant que nous sommes.
Je me sers d'animaux pour instruire les hommes.
Illustre rejeton d'un Prince aimé des Cieux,
Sur qui le monde entier a maintenant les yeux,
Et qui, faisant fléchir les plus superbes têtes,
Comptera désormais ses jours par ses conquêtes,
Quelque autre te dira d'une plus forte voix
Les faits de tes aïeux et les vertus des rois.
Je vais t'entretenir de moindres aventures,
Te tracer en ces vers de légères peintures :
Et, si de t'agréer je n'emporte le prix,
J'aurai du moins l'honneur de l'avoir entrepris.
 
 
PRÉFACE
 
      L'indulgence que l'on a eue pour quelques-unes de mes fables me donne lieu d'espérer la même grâce pour ce recueil. Ce n'est pas qu'un des maîtres de notre éloquence n'ait désapprouvé le dessein de les met

### Helpers

Define some methods to read this text
- a batch generator, generating batchs of text
- a decoder to translate a batch into stg more convinient

In [9]:
import numpy as np

vocab = sorted(set(text))  # my vocabulary (many letters)
print("I have", len(vocab)), "different elements in  my text which are :"
print (' '.join(vocab))


def sample_gen(batch_size, n_items):
    """Return a random sample"""
    while True:
        permutations = list(np.random.permutation(len(text) - n_items))
        while len(permutations) > n_items + 1:
            # Generate a batch
            batch = []
            for i in range(batch_size):
                p = permutations.pop()
                batch.append(text[p : p + n_items])
            yield batch

def encode_batch(batch, one_hot=False):
    """Takes a batch of string as input and encode it to a numerical
    batch"""
    batch_new = np.ndarray((len(batch),len(batch[0])))
    for i in range(len(batch)):
        for j in range(len(batch[0])):
            batch_new[i][j] = vocab.index(batch[i][j])
    
    if one_hot == True:
        # One hot the vector
        # TODO
        pass
    return batch_new


a = sample_gen(10, 28)
b = a.__next__()
print(b)
print(encode_batch(b))

I have 90

   ! " ' ( ) , - . 0 1 2 3 4 5 6 7 8 9 : ; ? A B C D E F G H I J L M N O P Q R S T U V X Y Z ` a b c d e f g h i j l m n o p q r s t u v x y z À Â Ç É Ê Ô à â ç è é ê ë î ï ô ù û
["  La raison m'est chose inco", 're discours.\nEnfin se sentan', ' et de meilleur débit,\nOn lu', "la montrent ;\nÀ l'égard de l", 'ttendent tout leur prix :\n  ', "'arrange.\nJe crois voir Anni", 'e à la bataille.\n" Ce bloc e', "pique d'être\n      Soigneux,", ' contre leur puissance. Quel', "x esprits\n      N'entendent "]
[[  1.   1.  33.  48.   1.  64.  48.  56.  65.  61.  60.   1.  59.   4.
   52.  65.  66.   1.  50.  55.  61.  65.  52.   1.  56.  60.  50.  61.]
 [ 64.  52.   1.  51.  56.  65.  50.  61.  67.  64.  65.   9.   0.  27.
   60.  53.  56.  60.   1.  65.  52.   1.  65.  52.  60.  66.  48.  60.]
 [  1.  52.  66.   1.  51.  52.   1.  59.  52.  56.  58.  58.  52.  67.
   64.   1.  51.  82.  49.  56.  66.   7.   0.  36.  60.   1.  58.  67.]
 [ 58.  48.   1.  59.  61.  60.  66.  64.  52.

### Sample of training taken from the web

In [24]:
# https://github.com/aymericdamien/TensorFlow-Examples/blob/master/notebooks/3_NeuralNetworks/recurrent_network.ipynb
import tensorflow as tf
from tensorflow.contrib import rnn
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

# Training Parameters
learning_rate = 0.001
training_steps = 100
batch_size = 128
display_step = 200

# Network Parameters
num_input = len(vocab)
timesteps = 28 # timesteps
num_hidden = 128 # hidden layer num of features
num_classes = len(vocab)

# tf Graph input
tf.reset_default_graph()
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])

# Define weights
W1 = tf.Variable(tf.random_normal([num_hidden, num_classes]))
B1 = tf.Variable(tf.random_normal([num_classes]))

def RNN(x, W1, B1):
    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], W1) + B1

with tf.name_scope('model'):
    logits = RNN(X, W1, B1)
    prediction = tf.nn.softmax(logits)

with tf.name_scope('loss'):
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
        logits=logits, labels=Y))

with tf.name_scope('optimizer'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op)

with tf.name_scope('metrics'):
    correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, training_steps+1):
        batch_x, batch_y = mnist.train.next_batch(100)

        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")



NameError: name 'training_steps' is not defined

### Train a model (6/10)
Train a model that can learn to create text from a given input (letter wise)

Dont forget to explain what you do, why, and if it do look to be working

## Explanations:
Before knowing that we will do this lab, we had to choose a subject for our HADOOP project. Therefore, Jean INGOLD, Natan ZIEGELEMEYER and I have decided to work on NLP by trying to generate some Baudelaire's texts after having trained the model.
So, here is our code and explanation that we took from our project.
Note that most of the code come from a French Google's phD, Mr Martin Gorner : https://github.com/martin-gorner/tensorflow-rnn-shakespeare.
He also made a video that really helped us to figure out how the algorithm works.


In our case, we could not use regular RNN because we strongly need to consider long-term dependencies.
LSTM could work but because it has 3 gates, it makes the computation too long.
Therefore, we are using GRU (Gated Recurrent Network) because it has 2 gates instead of 3, so it makes the computation faster.


•	CellSize: internal size of our GruCell. This is a free parameter, meaning that we can choose the value we want (512 in our case) <br>
•	NLAYERS: number of layers in our network. In our example, we have 3 layers. <br>
•	SEQLEN: the length of the sequence of our input (X). We have decided to proceed the training with a sequence of 30 characters.<br>
•	ALPHASIZE: Specification of the supported alphabet (ASCII-7)<br>
•	BATCHSIZE: the size of our batch. More precisely, it is the number of samples we are going to propagate through our network. In our code, the value is 100.<br>
•	EPOCH: number of times a training sample is used to update its weights. In our case, the value is 12.<br>

The GRU (Gated Recurrent Unit) has very similar equations than LSTM, the only difference is that it has only 2 gates instead of 3 so it makes it cheaper to compute. So, that’s what we will use!
![title](img/1.png)
![title](img/11.png)

With this simple tensorflow’s command, we create our GRU cell. It also defines the weigths and biases internally. The only parameter we need to specify is the internally size of our GRU cell. It is a free parameter and in our example we choose the value 512.
![title](img/2.png)
Let’s say that now we want a deep neural network. We use the command, above, and we only need to specify how many layers we want, so in our example we have 3 layers.
![title](img/22.png)

And now the last step we need to do is to unroll it using the command below:
![title](img/3.png)
![title](img/33.png)

How many times do we unroll? It depends on the shape of X. In other words, it depends on how many characters we want to consider for each sequence of X.
In the example above, we consider 8 characters, but in our code, we actually put the value as 20. It makes the training more accurate, and by the way we have more than 479 387 characters to train so 20 is the least.
The term “dynamic” used is the command means that using Tensorflow, we are able to unroll different times dynamically. That’s really powerful, but in our case, it is not relevant because we are always unrolling the same number of times.

•	For the output Hr, we actually need to compute a softmax activation layer on those to obtain my result characters
•	For H, they are our final states that we will need to pass back at the beginning as we are continuing our training model.

As shown previously in the schema, the output here is Hr. Hr has as shape [SEQLEN, CELLSIZE] because it depends on how many times we unrolled the cells and on the internal size of the GRU cell.
But in order to compute our softmax layer, we actually need to reshape Hr. Basically, all what we need to do is to combine all the Hr’s in a big bag. Let’s call Hf the result of this reshape, so its shape is now [BATCHSIZE x SEQLEN, CELLSIZE]. This reshape is useful because once we process this softmax layer, it means that we treat one of this Hr that comes from a new iteration of the cell or because it comes from one of the batch of our samples. So, it allows us to treat those H outputs exactly the same.
![title](img/y_b.png)

Then we just need to add a softmax activation layer with ALPHASIZE as parameter so the shape is now [BATCHSIZE x SEQLEN, ALPHASIZE]. So, for each sequence for each batch, each character has its own argument in the ALPHA sequence.
![title](img/y_logits.png)

Afterwards, we are able to execute our softmax layer so we will get our predictions.
So, for each alpha value, we will get a probability. Then, we take the argument that got the highest probability, and this argument is actually the letter that will be the output.
![title](img/Yo.png)

Finally, because we now know our prediction and our ouput, we are able to compute the loss.
Now that we get the loss, we are ready to train, and so reduce the loss.
In our code, we used the AdamOptimizer that implements the Adam algorithm, a variant of the Gradient Descent.
![title](img/train_step.png)



![title](img/batches.png)
It is necessary to batch the information correctly. For example, let’s say that each batch can afford 6 characters. Once we arrive to the last character, we strongly have to pass the rest of the sentence to the next batch. It is necessary because as we have talked about at the beginning, the last state of my network will feed as input state the next network of GRU cells (next batch). This particularity is due to our objective that is to predict and generate a Baudelaire’s text. Otherwise, the text generated would not have any sense. In other words, the internal state is passed horizontally in the cells, no matters the batch we are computing on.




### Results

![title](img/results.png)

So even if we there are some mistake, it seems to be look like French.
From the first epoch, the results were not good. It was just random characters, and it did not make any word for any language.

We can notice that the structure is quite the same. Moreover, we can see that the program understand that the characters are animals (symbols of Jean de la Fontaine).

In [4]:
#https://github.com/martin-gorner/tensorflow-rnn-shakespeare. 

import tensorflow as tf
from tensorflow.contrib import layers
from tensorflow.contrib import rnn  # rnn stuff temporarily in contrib, moving back to code in TF 1.1
import os
import time
import math
import numpy as np
import my_txtutils as txt


tf.set_random_seed(1)


SEQLEN = 30
BATCHSIZE = 100
ALPHASIZE = txt.ALPHASIZE
INTERNALSIZE = 512
NLAYERS = 3
learning_rate = 0.001  # fixed learning rate
dropout_pkeep = 0.8    # some dropout


# load data, here all the text are in the same .txt file (=fables)
shakedir = "fables.txt"
codetext, valitext, bookranges = txt.read_data_files(shakedir, validation=True)


# display some stats on the data
epoch_size = len(codetext) // (BATCHSIZE * SEQLEN)
txt.print_data_stats(len(codetext), len(valitext), epoch_size)


lr = tf.placeholder(tf.float32, name='lr')  # learning rate
pkeep = tf.placeholder(tf.float32, name='pkeep')  # dropout parameter
batchsize = tf.placeholder(tf.int32, name='batchsize')

# inputs
X = tf.placeholder(tf.uint8, [None, None], name='X')    # [ BATCHSIZE, SEQLEN ]
Xo = tf.one_hot(X, ALPHASIZE, 1.0, 0.0)                 # [ BATCHSIZE, SEQLEN, ALPHASIZE ]
# expected outputs = same sequence shifted by 1 since we are trying to predict the next character
Y_ = tf.placeholder(tf.uint8, [None, None], name='Y_')  # [ BATCHSIZE, SEQLEN ]
Yo_ = tf.one_hot(Y_, ALPHASIZE, 1.0, 0.0)               # [ BATCHSIZE, SEQLEN, ALPHASIZE ]

# input state
Hin = tf.placeholder(tf.float32, [None, INTERNALSIZE*NLAYERS], name='Hin')  # [ BATCHSIZE, INTERNALSIZE * NLAYERS]

# using a NLAYERS=3 layers of GRU cells, unrolled SEQLEN=30 times
# dynamic_rnn infers SEQLEN from the size of the inputs Xo

# How to properly apply dropout in RNNs: see README.md
cells = [rnn.GRUCell(INTERNALSIZE) for _ in range(NLAYERS)]
#Generate 3 grucells of 512 units


# "naive dropout" implementation
dropcells = [rnn.DropoutWrapper(cell,input_keep_prob=pkeep) for cell in cells]
multicell = rnn.MultiRNNCell(dropcells, state_is_tuple=False)
multicell = rnn.DropoutWrapper(multicell, output_keep_prob=pkeep)  # dropout for the softmax layer

Yr, H = tf.nn.dynamic_rnn(multicell, Xo, dtype=tf.float32, initial_state=Hin)
# Yr: [ BATCHSIZE, SEQLEN, INTERNALSIZE ]
# H:  [ BATCHSIZE, INTERNALSIZE*NLAYERS ] # this is the last state in the sequence

H = tf.identity(H, name='H')  # just to give it a name

# Softmax layer implementation:
# Flatten the first two dimension of the output [ BATCHSIZE, SEQLEN, ALPHASIZE ] => [ BATCHSIZE x SEQLEN, ALPHASIZE ]
# then apply softmax readout layer. This way, the weights and biases are shared across unrolled time steps.
# From the readout point of view, a value coming from a sequence time step or a minibatch item is the same thing.

Yflat = tf.reshape(Yr, [-1, INTERNALSIZE])    # [ BATCHSIZE x SEQLEN, INTERNALSIZE ]
Ylogits = layers.linear(Yflat, ALPHASIZE)     # [ BATCHSIZE x SEQLEN, ALPHASIZE ]
Yflat_ = tf.reshape(Yo_, [-1, ALPHASIZE])     # [ BATCHSIZE x SEQLEN, ALPHASIZE ]
loss = tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits, labels=Yflat_)  # [ BATCHSIZE x SEQLEN ]
loss = tf.reshape(loss, [batchsize, -1])      # [ BATCHSIZE, SEQLEN ]
Yo = tf.nn.softmax(Ylogits, name='Yo')        # [ BATCHSIZE x SEQLEN, ALPHASIZE ]
Y = tf.argmax(Yo, 1)                          # [ BATCHSIZE x SEQLEN ]
Y = tf.reshape(Y, [batchsize, -1], name="Y")  # [ BATCHSIZE, SEQLEN ]

train_step = tf.train.AdamOptimizer(lr).minimize(loss)



# stats for display

seqloss = tf.reduce_mean(loss, 1)
batchloss = tf.reduce_mean(seqloss)
accuracy = tf.reduce_mean(tf.cast(tf.equal(Y_, tf.cast(Y, tf.uint8)), tf.float32))
loss_summary = tf.summary.scalar("batch_loss", batchloss)
acc_summary = tf.summary.scalar("batch_accuracy", accuracy)
summaries = tf.summary.merge([loss_summary, acc_summary])

# Init Tensorboard stuff. This will save Tensorboard information into a different
# folder at each run named 'log/<timestamp>/'. Two sets of data are saved so that
# you can compare training and validation curves visually in Tensorboard.

timestamp = str(math.trunc(time.time()))
summary_writer = tf.summary.FileWriter("log/" + timestamp + "-training")
validation_writer = tf.summary.FileWriter("log/" + timestamp + "-validation")

# for display: init the progress bar

DISPLAY_FREQ = 50
_50_BATCHES = DISPLAY_FREQ * BATCHSIZE * SEQLEN
progress = txt.Progress(DISPLAY_FREQ, size=111+2, msg="Training on next "+str(DISPLAY_FREQ)+" batches")

# init
istate = np.zeros([BATCHSIZE, INTERNALSIZE*NLAYERS])  # initial zero input state
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
step = 0

# training loop
for x, y_, epoch in txt.rnn_minibatch_sequencer(codetext, BATCHSIZE, SEQLEN, nb_epochs=12):

    # train on one minibatch
    feed_dict = {X: x, Y_: y_, Hin: istate, lr: learning_rate, pkeep: dropout_pkeep, batchsize: BATCHSIZE}
    _, y, ostate = sess.run([train_step, Y, H], feed_dict=feed_dict)

    # log training data for Tensorboard display a mini-batch of sequences (every 50 batches)
    if step % _50_BATCHES == 0:

        feed_dict = {X: x, Y_: y_, Hin: istate, pkeep: 1.0, batchsize: BATCHSIZE}  # no dropout for validation
        y, l, bl, acc, smm = sess.run([Y, seqloss, batchloss, accuracy, summaries], feed_dict=feed_dict)
        txt.print_learning_learned_comparison(x, y, l, bookranges, bl, acc, epoch_size, step, epoch)
        summary_writer.add_summary(smm, step)


    # run a validation step every 50 batches
    # The validation text should be a single sequence but that's too slow (1s per 1024 chars!),
    # so we cut it up and batch the pieces (slightly inaccurate)
    # tested: validating with 5K sequences instead of 1K is only slightly more accurate, but a lot slower.

    if step % _50_BATCHES == 0 and len(valitext) > 0:

        VALI_SEQLEN = 1*1024  # Sequence length for validation. State will be wrong at the start of each sequence.
        bsize = len(valitext) // VALI_SEQLEN
        txt.print_validation_header(len(codetext), bookranges)
        vali_x, vali_y, _ = next(txt.rnn_minibatch_sequencer(valitext, bsize, VALI_SEQLEN, 1))  # all data in 1 batch
        vali_nullstate = np.zeros([bsize, INTERNALSIZE*NLAYERS])
        feed_dict = {X: vali_x, Y_: vali_y, Hin: vali_nullstate, pkeep: 1.0,  # no dropout for validation
                     batchsize: bsize}

        ls, acc, smm = sess.run([batchloss, accuracy, summaries], feed_dict=feed_dict)
        txt.print_validation_stats(ls, acc)

        # save validation data for Tensorboard
        validation_writer.add_summary(smm, step)



    # display a short text generated with the current weights and biases (every 150 batches)

    if step // 3 % _50_BATCHES == 0:
        txt.print_text_generation_header()
        ry = np.array([[txt.convert_from_alphabet(ord("K"))]])
        rh = np.zeros([1, INTERNALSIZE * NLAYERS])

        for k in range(1000):

            ryo, rh = sess.run([Yo, H], feed_dict={X: ry, pkeep: 1.0, Hin: rh, batchsize: 1})
            rc = txt.sample_from_probabilities(ryo, topn=10 if epoch <= 1 else 2)
            print(chr(txt.convert_to_alphabet(rc)), end="")
            ry = np.array([[rc]])

        txt.print_text_generation_footer()

    # display progress bar
    progress.step(reset=step % _50_BATCHES == 0)

    # loop state around
    istate = ostate
    step += BATCHSIZE * SEQLEN





Loading file fables.txt
Training text size is 0.46MB with 0.00KB set aside for validation. There will be 159 batches per epoch

   0 (epoch 0) fables.txt │   Monseigneur le Dauphin\ \Je  │                                │ loss: 4.46466
  30 (epoch 0) fables.txt │ excit  les autres   mieux fair │                                │ loss: 4.45685
  60 (epoch 0) fables.txt │  sommes l'abr g  de ce qu'il y │                                │ loss: 4.43763
  90 (epoch 0) fables.txt │ sonnages qui ont le mieux m ri │                                │ loss: 4.45575
 120 (epoch 0) fables.txt │ raitements seraient sus. Z nas │                                │ loss: 4.47748
 150 (epoch 0) fables.txt │ 'accommod rent. On ne parla pl │                                │ loss: 4.49244
 180 (epoch 0) fables.txt │  disant que la langue est la p │                                │ loss: 4.41396
 210 (epoch 0) fables.txt │ ent ainsi, il y consentait ; p │                                │ loss: 4.44082
 240 (ep

upnl u immeiiemttr esrrr'enJinnrlJ olestto's 'ilte i eJs   ulenor lie rJus'lulpr rnJsieinm enesi l rroirs osourosnlolu slioeuui oounmuoolle loelitls'e'o--l-eulneonleJns iol nil loiseiiiln nloi oe'nli Jenuli le essssnlr n l siuiJii 'eepe'nlrlu eueiJi  ee  ermsl sieuson srei eu emoullilr ueeJini''iJssesselum noe suni  ioors'oonnsoneuousJe' uiuemJ'nsnJu  msmrs-resJ nreJsui ls'res elsml n oiio ol inr   e Jelleess iJ'noueloeorJr s e lir'r' s   enmnlll   olutonitnonenietJnl eJinslJ luir e'isnr rJisilesurnimumuu s ul erssJinnes nus emn eesl rooooJl ' u  ' se nulseuleru e uols uJrn'isuoiJulnni-- m le-iu-e -nrnieuuueu uum! !iim u lenmmllruu on- -rlneu-ill  neuestn -unoin suoJsseiooulel   ml Je''soluuuoJloi reinoe' 'nuo oJlJ'' i'o l -s n  JJuorer s s rsrssseol on looulesu ts toulllei ue'uiuuiimmrGelsrmr-reinelieiu  eJeei  sl iueroonrsJ s nouneeenu J'leeueJ 'uloooluueslsei niuslsoun lm souuioomtruisesrl   neneusi on l le n sJuieioilre lsoeluunn luoJseJJunl 'nlis-Jine sil lnee eere' uo ssei o  n' 

300000 (epoch 0) fables.txt │ sie sans harmonie ; mais il n' │  e   e  eertee      er  ee  ee │ loss: 2.80657
300030 (epoch 0) fables.txt │ rtu ; plut t que d' tre r duit │ e e   ee  e eee  eee e  eeee   │ loss: 2.84299
300060 (epoch 0) fables.txt │  ses fables. Cela m'a paru d'a │      eree  \ e  e eeeeeite eee │ loss: 3.03418
300090 (epoch 0) fables.txt │ rades ne parurent point  tonn  │ eee   e  ertee   eert  ee e    │ loss: 2.57003
300120 (epoch 0) fables.txt │ n des deux, il devait donner   │   e   e     e  e ee  eee   eee │ loss: 2.63234
300150 (epoch 0) fables.txt │ l acheta force gibier, comme p │   eee  e ei e  etee e  eiee  e │ loss: 2.83055
300180 (epoch 0) fables.txt │ j  la cervelle, aussi bien au  │ ee ee e  e  e   e   e ee  eet  │ loss: 2.76887
300210 (epoch 0) fables.txt │ e retour, Xantus commanda que  │   e  et  eei  e eereee ee ee   │ loss: 2.76738
300240 (epoch 0) fables.txt │ s'envoyaient les uns aux autre │ ee  ee ee    e   e  eet eet e  │ loss: 2.64229
300270 (ep

450000 (epoch 0) fables.txt │ que j'ai choisies, et si ce to │ ue leentleent nn \en pe se seu │ loss: 2.00707
450030 (epoch 0) fables.txt │ jugement et les moeurs, on se  │ eiene   dn le  eeusr   en se p │ loss: 2.15536
450060 (epoch 0) fables.txt │  n cessaires que celle-l . Tan │ de e   nte  due le  e se \"eut │ loss: 2.38328
450090 (epoch 0) fables.txt │ ause qu'il changea de ma tre.  │ nt  dueen eeente nde leitee \  │ loss: 2.23262
450120 (epoch 0) fables.txt │ ; qu'il y avait longtemps que  │ \ ueen eeuneit deu ee ee due l │ loss: 2.30353
450150 (epoch 0) fables.txt │ tte du premier de tous les dev │    de poesen  ee leus de  de e │ loss: 1.93696
450180 (epoch 0) fables.txt │   tait vaincu, et demanda pard │   eit deit e  en le eitendeite │ loss: 2.31021
450210 (epoch 0) fables.txt │  s'il disait mieux que le ma t │ eean ee  nt de n  eue le peite │ loss: 2.09346
450240 (epoch 0) fables.txt │ eura court, ce qui fit que le  │ n  ndeus   ee sue let eue le p │ loss: 1.89684
450270 (ep

     Lonts urr t  a da pr tie. Int cataias l'uirt de sansse de sais den d  ceses araice
      L'imen den moss de cercoustriter,Qert d  fes.
          Le s  pet es m nt de sopmeste
         Et seusere pramrons le lois e       Lri laris levanine.
  
     Aust uandeunes mrinn ui s me titre,.    Lan u  dairin p ure de damirre le  l'itre de m'uin chals, ses don uaitare :
          Au leritire lerts sonnr de tet  elat cat  ne sous de l s, etse, et ques  siu derent aur samest le phenait les cesceur se ttain :
Aene fous eule canter doit, lomne sir,.       Et div s sos das peutre lovtit,
Le v s eus l'ille le dontiit an de d'enssse.
      Douse l'aivou, en        Quecez la duis ain copti en laren  tren,
     Le moussons l'ei de de vent un sairon es cos ent de las llons an aicne :
"      El savis dut us alas ue mense,
      Nestonre peumle ttes latts peui,
 n      N'e callaicire me lien des, mes, mit pest et son meui,, toun seiprent. 
└───────────────────────────────────────────────End of generat

...

      L'autre serait pour sans ces pares.
      Le Lion des plaissants, l'heure en peut la plupagnie
      D'un mort qu'il avait fait de l'argent.
      Le Lion de sa bourse en passait :
      Il la pauva traiter, et l'autre prise ;
Et qu'il ne pouvait pas, dit le Doup de l'autre.
 
Livre second Fable 18
 
 
Le Chier et le Roi
 
      Un Chat de Chats ! "
      Le Lion de sa bite avec son Fil,
      Si j'est pas un servir la bonne ;
      Il n'avait pas un par sa marche :
      Il n'est pas par le plus pas au lain.
      Il n'a pas par le troupeau de l'autre ;
      Et l'on voulait pas l'autre le croire,
      Et qui sais que le pauvre Amare
      Se sans pas traiter, et le plus propos :
" Mais ce sont par les plains ; car, sous le propos,
      Et le prix par son souper la place
D'autres dertiers sort de l'autre au prix de la sorte ;
      Et que le Chat, de la chose avait partir :
" Mais je vous pris pas l'autre par le propos des prenis
└──────────────────────────────────────────────

5250000 (epoch 11) fables.txt │ ur du Corbeau.\Que vous  tes j │ r,de torbeau \ ue dous ntes de │ loss: 1.15537
5250030 (epoch 11) fables.txt │ , je pr tends la troisi me.\Si │  de neitends de peous  me \ o  │ loss: 1.07522
5250060 (epoch 11) fables.txt │ ts en campagne aussit t ;\Et l │ o nn serpagne eutsi  t d\ t qe │ loss: 1.08882
5250090 (epoch 11) fables.txt │ r survient, qui les accorde ne │  der itnt  due ne  aucoude :o  │ loss: 1.38908
5250120 (epoch 11) fables.txt │ fagot, il songe   son malheur  │ aiet  el seute d lon sa  eur : │ loss: 1.28929
5250150 (epoch 11) fables.txt │ s Frelons et les Mouches   mie │  deemons et le  Couthes\e loeu │ loss: 1.15251
5250180 (epoch 11) fables.txt │ n Corbeau sur sa voix.\De plus │  shrbeau der lo piix.\ e ceus  │ loss: 1.15967
5250210 (epoch 11) fables.txt │ se retirer.\      Deux jours a │ enpetore         Le x fours du │ loss: 1.27456
5250240 (epoch 11) fables.txt │       Parlait un jour au Mouch │      Dor e t ln pour du couthe │ loss: 

5400000 (epoch 11) fables.txt │ voulu pour beaucoup en  tre so │ ousu pour soaucoup dn pcae eon │ loss: 1.13584
5400030 (epoch 11) fables.txt │ re oeil qu'on ne voit son proc │ e cuul euiin le sout pau paiph │ loss: 1.24304
5400060 (epoch 11) fables.txt │ Le Loup l'emporte, et puis le  │ e poup deanporte  et lais le p │ loss: 0.88476
5400090 (epoch 11) fables.txt │ mpagnie. "\Simonide promit. Pe │ pagnee,\"\ e pn ee daomet  Lon │ loss: 1.60999
5400120 (epoch 11) fables.txt │    son tour.\Toutes deux firen │ d lon srmr \ out s deux folent │ loss: 1.04409
5400150 (epoch 11) fables.txt │ qu'on nous mange, on nous grug │ u'in leus fonqe  en peus prose │ loss: 1.41441
5400180 (epoch 11) fables.txt │ , et plus t t que plus tard,\A │  et daus dot du' laus donde\ u │ loss: 1.39940
5400210 (epoch 11) fables.txt │ i demande encore une quinzaine │  dimande en or  en  fuettente  │ loss: 1.44230
5400240 (epoch 11) fables.txt │  p rils tel a pu se soustraire │ setits do  evtaiserponv rente, │ loss: 

      Et disait-il, et par la vie,
      Et l'avertir en sa porte au point. " Le Loup, les prenier de l'autre ;
      Et le peu laissait pour les propos,
      Et de la saison de la voix.
      J'en avais de la charte de son ami.
      L'autre de contre l'homme et les plaisers,
      Le soupine des Doeux et les voisins ;
      En cette fort de ces contes des voleurs, et dit le Loup par sa voix,
      Et, se plus grands de son ambre.
      L'air des Dieux de la vie en son ami,
      Les gens de cette affaire :
      Ils sont par son sagn de la voix.
      L'un de l'autre en voit de l'aventure,
      Les gens, dit l'heure en son sage,
Le plaisir des Dieux de l'argent et de son sang de sa vie.
      Le Loup, pour son pare et sa fortune.
Le Lion des Dieux, en souvant les vers, et les plaisirs,
      Les consid res d'un temps des plus grands sommes,
      Et par les prendre et les compagnons ;
      Et de ces petits et l'autre se retirent ;

└───────────────────────────────────────────────E

5700000 (epoch 11) fables.txt │ premi re en qualit  de Sire :\ │ aoniere en cuelit  de lone :\  │ loss: 1.14701
5700030 (epoch 11) fables.txt │ s deux amis.\ \Le r gal fut fo │  meux euis \  Livresar dut par │ loss: 1.25759
5700060 (epoch 11) fables.txt │ efois une pauvre province.\    │ lons ln  cervre haofince\\     │ loss: 1.08967
5700090 (epoch 11) fables.txt │ .\ \ \La Mort et le B cheron\  │    L Li Mort es le Rother n\ \ │ loss: 1.07116
5700120 (epoch 11) fables.txt │ t-il ;\Mais le moindre grain d │ -il,;  ais ie moinsre eronn de │ loss: 1.03860
5700150 (epoch 11) fables.txt │ aya la constance et la peine.  │ rsile pousiince dn le paune \" │ loss: 1.39514
5700180 (epoch 11) fables.txt │ \Moi Souris ! des m chants vou │  an qouris d dis aomhants dous │ loss: 1.37000
5700210 (epoch 11) fables.txt │ e autre saison, quand la race  │  futre fouson.\luend ie maie e │ loss: 1.07840
5700240 (epoch 11) fables.txt │ te.\Tandis qu'  le tuer mon vi │ e,\ ont s luei la prmrrdon toe │ loss: 

### Train a model (4/10)
Train a model that can learn to create text from a given input (text wise). Using a word embeding seen in class, like CBOW

Dont forget to explain what you do, why, and if it do look to be working

In [2]:
import tensorflow as tf
from tensorflow.contrib import layers
from tensorflow.contrib import rnn  # rnn stuff temporarily in contrib, moving back to code in TF 1.1
import os
import time
import math
import numpy as np
import my_txtutils as txt


tf.set_random_seed(1)


SEQLEN = 30
BATCHSIZE = 100
ALPHASIZE = txt.ALPHASIZE
INTERNALSIZE = 512
NLAYERS = 3
learning_rate = 0.001  # fixed learning rate
dropout_pkeep = 0.8    # some dropout


# load data, here all the text are in the same .txt file (=fables)
shakedir = "fables.txt"
codetext, valitext, bookranges = txt.read_data_files(shakedir, validation=True)


# display some stats on the data
print(len(codetext))


Loading file fables.txt
479387
